In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import PolynomialFeatures, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split 

from sklearn.preprocessing import PolynomialFeatures

from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor\

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

In [2]:
# Путь к тренировочному набору
path_train = 'train.csv'# содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)


In [3]:
X = df_train.drop(columns='priceUSD')
y = df_train.priceUSD

In [4]:
cat_features=['company', 'model', 'condition', 'fuel', 'color', 'transmission', 'drive_unit', 'vehicle_size_class']
num_features=['year', 'mileage(km)', 'volume(cm3)']

In [5]:
num_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

cat_transformer = Pipeline(
    steps=[('ce', ce.OrdinalEncoder()),
           ('scaler', MinMaxScaler())]
)

size_map = [{
    'col': 'vehicle_size_class',
    'mapping': {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'S': 7, 'M': 8, 'J': 9}
}]

clarity_transformer = Pipeline(steps=[
    ('missing_num', SimpleImputer(strategy='most_frequent')),
    ('ce', ce.OrdinalEncoder(mapping=size_map)),
    ('scaler', MinMaxScaler())
])    


CT = ColumnTransformer([
       ("num", num_transformer, num_features),
    ("cat", cat_transformer, cat_features),
        ("ordinal_map", clarity_transformer, ['vehicle_size_class'])
        ]).set_output(transform='pandas')

display(CT)
ct = CT.fit_transform(X)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['year', 'mileage(km)', 'volume(cm3)']),
                                ('cat',
                                 Pipeline(steps=[('ce', OrdinalEncoder()),
                                                 ('scaler', MinMaxScaler())]),
                                 ['company', 'model', 'condition', 'fuel',
                                  'color', 'transmission', 'drive_unit',
                                  'vehicle_size_class']),
                                ('ordinal_map',
                                 Pipeline(steps=[('missing_num',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ce',
                                                  OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                           'data_type': dtype('O'),
                                                                           'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                 ('scaler', MinMaxScaler())]),
                                 ['vehicle_size_class'])])

In [6]:
pipe = Pipeline([
                ('model', TransformedTargetRegressor(
                        regressor = Pipeline(steps=[
                                ('preproc', CT),
                                ('estimator',  RandomForestRegressor(n_estimators=1000,criterion='friedman_mse'))
                            ]),
                        transformer=PowerTransformer(),
                        check_inverse = False)
                )])


pipe.fit(X, y)

Pipeline(steps=[('model',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=Pipeline(steps=[('preproc',
                                                                       ColumnTransformer(transformers=[('num',
                                                                                                        Pipeline(steps=[('scaler',
                                                                                                                         StandardScaler())]),
                                                                                                        ['year',
                                                                                                         'mileage(km)',
                                                                                                         'volume(cm3)']),
                                                                                                       ('cat',
                                                                                                        Pipeline(steps=[('ce',
                                                                                                                         OrdinalEncoder()),
                                                                                                                        ('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['company',
                                                                                                         'model',
                                                                                                         'condition',
                                                                                                         'fuel',
                                                                                                         'colo...
                                                                                                        Pipeline(steps=[('missing_num',
                                                                                                                         SimpleImputer(strategy='most_frequent')),
                                                                                                                        ('ce',
                                                                                                                         OrdinalEncoder(mapping=[{'col': 'vehicle_size_class',
                                                                                                                                                  'data_type': dtype('O'),
                                                                                                                                                  'mapping': A    1
B    2
C    3
D    4
E    5
F    6
S    7
M    8
J    9
dtype: int64}])),
                                                                                                                        ('scaler',
                                                                                                                         MinMaxScaler())]),
                                                                                                        ['vehicle_size_class'])])),
                                                                      ('estimator',
                                                                       RandomForestRegressor(criterion='friedman_mse',
                                                                                             n_estimators=1000))]),
                                            transformer=PowerTransformer()))])

In [7]:
y_predicted = pipe.predict(df_test)

In [8]:
y_predict = y_predicted
y_predict

array([11067.49620799,  8945.00254212,  2253.73023357, ...,
       10405.10959624,  4517.67864946,  9014.83028121])